### 수집된 포털 정보기반 딥크롤링
* 기존에 수집된 정보를 바탕으로 매체별 크롤링
* 다양한 서브 데이터의 정형화
* URL 기반의 다양한 크롤링 기법 사용

### 분석요소
* 제목 :
* 내용 : 
* 작성일 : 
* 매체 : 
* 작성자 :

### 기존 수집 데이터 활용

In [1]:
url='https://search.naver.com/search.naver?&where=news&query={0}&start={1}'.format('신세계',11)
url

'https://search.naver.com/search.naver?&where=news&query=신세계&start=11'

In [9]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib
import matplotlib.pyplot as plt
import sqlite3
import re
from datetime import datetime, timedelta,date
import sys

In [10]:
def calcDate(txt=''):
    #print(txt)
    rdate=''
    nowd=datetime.now()
    pattern='[^\d]'
    num=int(re.sub(pattern,'',txt))
    if('일' in txt):
        res=nowd + timedelta(days=num*-1)
    elif('.' in txt):
        ds=txt.replace('.','-')
        #print(ds)
        res=ds+' 00:00:00.000000'
    else:
        res=nowd + timedelta(hours=num*-1)
    return res

In [11]:
calcDate('2021.2.1')

'2021-2-1 00:00:00.000000'

In [12]:
def makeKey(txt=''):
    txt_=re.sub('http\S*//',' ',txt)
    key=re.sub('[^a-zA-Z0-9%]','',txt_)
    return key

In [40]:
def getData(whr=''):
    dbname='navernews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select ncomp,nurl from nnews '+whr
        res=cursor.execute(sql).fetchall()
    return(res)

In [42]:
allurl = getData()
allurl[:5]

[('연합뉴스', 'http://yna.kr/AKR20210317071000030?did=1195m'),
 ('이데일리',
  'http://www.edaily.co.kr/news/newspath.asp?newsid=01430086628984304'),
 ('이투데이', 'https://www.etoday.co.kr/news/view/2005423'),
 ('파이낸셜뉴스', 'http://www.fnnews.com/news/202103180822274935'),
 ('뉴스1', 'https://www.news1.kr/articles/?4244392')]

In [14]:
def insSub(d):
    dbname='navernews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from subnews where nkey="'+d[5]+'"'
        res=cursor.execute(sql).fetchall()
        if(len(res)>0):
            print("이미 존재 합니다.:"+d[1])
        else:
            sql='insert into subnews (ncate,ntitle,ncomp,ndate,nurl,nkey) values(?,?,?,?,?,?)'
            res=cursor.execute(sql,d)
            print('.',end='')
        

In [54]:
import requests
from bs4 import BeautifulSoup

In [63]:
# res=requests.get(url)
# src=res.content.strip()
# htmls=bsp(src,'html.parser')

# r = requests.get(url)
# soup = BeautifulSoup(r.text,'html.parser')
# title = soup.select_one('articleWrap > div.content03 > header > h1')


title_block=htmls.select('div.content03')
title = title_block.select('h1.tit').text.strip()
print(title)

AttributeError: 'list' object has no attribute 'select'

In [8]:
def getSub(url):
    res=requests.get(url)
    src=res.content.strip()
    htmls=bsp(src,'html.parser')
    print(url)
    title_block=htmls.select('header.title-article01 > h1.tit')
    printnt(title_block)
    if(len(tgts)==0):
        print('데이터가 없습니다.'+url)
        sys.exit(1)
    for tgt in tgts:
        mainarti=tgt.select('div.news_wrap')
        subartis=tgt.select('ul.list_cluster > li.sub_bx')
        if(len(mainarti)>0):
            title_= mainarti[0].select('a.news_tit')[0].text
            title=re.sub('[^ㄱ-힣0-9 a-zA-Z]',' ',title_)
            comp=mainarti[0].select('.info.press')[0].text.replace('언론사 선정','')
            date_=mainarti[0].select('span.info')[0].text.replace('시간 전','')
            date=calcDate(date_)
            url=mainarti[0].select('a.news_tit')[0]['href']
            key=makeKey(url)
            opt=1
            d=(opt,title,comp,date,url,key)
            insMain(d)
            for subarti in subartis:
                title_= subarti.select('a.sub_tit')[0].text
                title=re.sub('[^ㄱ-힣0-9 a-zA-Z]',' ',title_)
                comp=subarti.select('.sub_txt.press')[0].text.replace('언론사 선정','')
                date_=subarti.select('span.sub_txt')[0].text.replace('시간 전','').replace('네이버뉴스','')
                date=calcDate(date_)
                url=subarti.select('a.sub_tit')[0]['href']
                key=makeKey(url)
                opt=2
                d=(opt,title,comp,date,url,key)
                insMain(d)



In [8]:
urlsrc='https://search.naver.com/search.naver?&where=news&query={0}&start={1}'
urlsrc='https://search.naver.com/search.naver?&where=news&query={0}&start={1}&pd=3&ds=2019.01.01&de=2019.12.31'
for i in range(54,100):
    url=urlsrc.format('신세계',i*10+1)
    print(url)
    getPage(url)

https://search.naver.com/search.naver?&where=news&query=신세계&start=541
https://search.naver.com/search.naver?&where=news&query=신세계&start=541
..................................https://search.naver.com/search.naver?&where=news&query=신세계&start=551
https://search.naver.com/search.naver?&where=news&query=신세계&start=551
이미 존재 합니다.:신세계아이앤씨  클라우드 매니지먼트 플랫폼  스파로스 CMP  출시
이미 존재 합니다.:신세계아이앤씨  클라우드매니지먼트플랫폼  스파로스 CMP  출시
이미 존재 합니다.:네이버 신세계  2500억원 규모 지분교환 결정
이미 존재 합니다.:네이버 자사주는 화수분   신세계 빅딜 에도 여전히 자사주 가치는 6조원  왜   
이미 존재 합니다.:이마트 신세계  네이버와 2500억 지분교환  전략적 제휴강화 
이미 존재 합니다.:이마트 신세계  네이버와 2천500억 지분 교환
이미 존재 합니다.:이마트 신세계  네이버와 지분교환
...이미 존재 합니다.:네이버 신세계그룹 맞손 2500억 지분 교환
이미 존재 합니다.:네이버 신세계그룹  상호 지분 교환 커머스 동맹 결성
이미 존재 합니다.:네이버 신세계  물류 커머스 기술 종합 협력     온 오프라인 시너지 
이미 존재 합니다.:네이버 신세계그룹  온오프라인 아우르는 새로운 커머스 경험 제공한다
..........이미 존재 합니다.:신세계백화점에서 만나는 프리미엄 육가공 브랜드  벨로타벨로타 
이미 존재 합니다.:신세계백화점 강남점  프리미엄 육가공 브랜드  벨로타벨로타  열어
이미 존재 합니다.:신세계 강남점  프리미엄 육가공 브랜드 벨로타벨로타 오픈
.https://search.naver.com/search.naver?&where=news&

https://search.naver.com/search.naver?&where=news&query=신세계&start=641
이미 존재 합니다.:이베이코리아 인수전에 SKT 롯데 신세계 등 참여 카카오 불참 종합2보 
.이미 존재 합니다.:신세계백화점  아동용품 인기 다음달 유모차 행사
이미 존재 합니다.:신세계백화점  3월 아동 매출 133     신학기 효과
이미 존재 합니다.:신세계百  아동용품 매출 고성장    3월 등교 봄날씨 영향 
이미 존재 합니다.:3월 등교 봄날씨에    신세계百   아동용품  매출 쑥
...이미 존재 합니다.:신세계푸드   올반 옛날통닭  10만개 판매 돌파
이미 존재 합니다.:신세계푸드  올반 옛날통닭  누적판매량 10만개 넘어
이미 존재 합니다.:신세계푸드  올반 옛날통닭  누적 판매량 10만개 돌파
이미 존재 합니다.:신세계푸드   올반 옛날통닭  높은 가성비 온라인 판매 通했다 
.이미 존재 합니다.: 속보  이마트 신세계  네이버와 2500억 지분 교환
..이미 존재 합니다.:신세계그룹 프로야구단  SSG랜더스   구단 공식 엠블럼   로고 공개
이미 존재 합니다.:신세계 그룹  신생팀 SSG 랜더스 엠블럼 공개  붉은색 계승 
이미 존재 합니다.:신세계 착륙한 우주선  SSG랜더스  야구단 엠블럼 확정
이미 존재 합니다.:신세계 야구단 SSG랜더스  공식 엠블럼 로고 공개
..https://search.naver.com/search.naver?&where=news&query=신세계&start=651
https://search.naver.com/search.naver?&where=news&query=신세계&start=651
이미 존재 합니다.:신세계그룹 프로야구단  SSG랜더스   구단 공식 엠블럼   로고 공개
이미 존재 합니다.:신세계 그룹  신생팀 SSG 랜더스 엠블럼 공개  붉은색 계승 
이미 존재 합니다.:신세계 착륙한 우주선  SSG랜더스  야구단 엠블럼 확정
이미 존재 합니다.:신세계 야구단 SSG랜더스 

https://search.naver.com/search.naver?&where=news&query=신세계&start=751
이미 존재 합니다.: NAVER  신세계와 지분스왑으로 차별화된 서비스 기대 
..이미 존재 합니다.: 부음  장혜진 신세계인터내셔날 상무 부친상
이미 존재 합니다.: 부고  장혜진 신세계인터내셔날 상무 부친상
이미 존재 합니다.: 부고  장혜진 신세계인터내셔날 상무 부친상
..이미 존재 합니다.:신세계  또 다시 재고면세 할인 판매   오프화이트도 풀린다
이미 존재 합니다.:신세계인터  명품 재고 푼다 11일부터  최대 40  할인
이미 존재 합니다.:신세계인터  반값 명품 판매 오프화이트도 선봬
이미 존재 합니다.:신세계인터   오프화이트  첫 재고 면세 판매
...이미 존재 합니다.: 부고  장혜진 신세계인터내셔날 상무 씨 부친상
이미 존재 합니다.: 부고  장혜진 신세계인터내셔날 상무  씨 부친상
이미 존재 합니다.: 부고 장혜진 신세계인터내셔날 상무  씨 부친상
이미 존재 합니다.:장혜진 신세계인터내셔날 상무 부친상
....이미 존재 합니다.:흑화당  대만 영양 샌드위치     신세계백화점 강남점서 사전 출시한다
..https://search.naver.com/search.naver?&where=news&query=신세계&start=761
https://search.naver.com/search.naver?&where=news&query=신세계&start=761
이미 존재 합니다.:신세계 강남점에 프랑스  벨로타벨로타  등장
이미 존재 합니다.: 오늘 저녁은 프리미엄 하몽  신세계百 강남점  프리미엄 육가공 브랜드 벨   
이미 존재 합니다.: 부고  장혜진 신세계인터내셔날 상무  씨 부친상
이미 존재 합니다.: 부고 장혜진 신세계인터내셔날 상무  씨 부친상
이미 존재 합니다.:장혜진 신세계인터내셔날 상무 부친상
이미 존재 합니다.: 부고  장혜진 신세계인터내셔날 상무  씨 부친상
.이미 존재 합니다.:신세계  1분기 영업익 전

https://search.naver.com/search.naver?&where=news&query=신세계&start=841
이미 존재 합니다.:신세계푸드  올반 옛날통닭 10만개 팔려 
이미 존재 합니다.:신세계푸드   올반 옛날통닭  누적판매량 10만개 돌파
이미 존재 합니다.:신세계푸드   올반 옛날통닭  누적판매량 10만개 돌파    온라인 채널 집중   
이미 존재 합니다.:신세계푸드   올반 옛날통닭  누적판매량 10만개 돌파 온라인 판매 박차
이미 존재 합니다.: 이제는 온라인이다  신세계푸드  올반 옛날통닭 누적판매량 10만개 돌파
이미 존재 합니다.:이번엔  오프화이트  풀린다 신세계 재고 명품 40  할인
이미 존재 합니다.:신세계면세점  오프화이트 등 명품 재고 최대 40  할인 실시
이미 존재 합니다.:신세계인터내셔날  11일부터 면세점 명품 재고 최대 40  할인
이미 존재 합니다.:신세계인터  에스아이빌리지서 신세계면세점 명품 재고 할인 판매
이미 존재 합니다.:신세계 시코르  새내기 신입사원 메이크업  꿀팁  배우세요 
이미 존재 합니다.:신세계백화점 뷰티 편집숍 시코르  랜선 뷰티 클래스 오픈
...이미 존재 합니다.:휠라코리아 케즈  한달새 신세계百 5개 점포 개점
이미 존재 합니다.:휠라코리아 케즈  신세계백화점 5개 점포에 매장 오픈
이미 존재 합니다.:케즈  신세계百 5개 점포 입점    소비자 접점 확대 
..이미 존재 합니다.: 속보 이마트 신세계  네이버와 2500억 지분 교환
...이미 존재 합니다.:신세계푸드 x 강릉초당두부  건강 디저트 케이크 출시
이미 존재 합니다.:신세계푸드  건강 디저트  강릉초당두부 케이크  출시
이미 존재 합니다.:신세계푸드   강릉초당두부  손잡고 건강 케이크 출시
이미 존재 합니다.:신세계푸드   강릉초당두부  협업 디저트 케이크 출시
.https://search.naver.com/search.naver?&where=news&query=신세계&start=851
https://searc

https://search.naver.com/search.naver?&where=news&query=신세계&start=921
이미 존재 합니다.:신세계 공식 발표  SK 와이번스의 새 이름은 SSG 랜더스 
이미 존재 합니다.:일렉트로스 or 랜더스  신세계 야구단 오늘 5일  이름표 달까
이미 존재 합니다.:신세계 이마트 야구단 새 이름은  SSG 랜더스 
이미 존재 합니다.:반전은 없었다 신세계야구단 명칭  예상대로  SSG 랜더스 
이미 존재 합니다.:야구 신세계 상륙작전    SSG랜더스 로 새 출발
이미 존재 합니다.:신세계ㆍ이마트 SSG랜더스  엠블럼 로고 공개  랜더스의 우주적 상륙   새로   
이미 존재 합니다.:신세계 야구단 SSG 랜던스  우주선 착륙 엠블럼 공개
이미 존재 합니다.:신세계 야구단  팀명  SSG LANDERS  확정
이미 존재 합니다.:신세계 야구단  새 팀명  SSG 랜더스 LANDERS  로 확정
이미 존재 합니다.:신세계 야구단 새 이름은  SSG 랜더스 LANDERS  
..이미 존재 합니다.: 포토뉴스  대구신세계  캘리포니아 감성 골프웨어  혼가먼트 
이미 존재 합니다.:디저트도 건강하게  신세계푸드   강릉초당두부 케이크  선봬
이미 존재 합니다.:신세계푸드   강릉초당두부  디저트 케이크 출시
이미 존재 합니다.:신세계푸드   강릉초당두부 와 디저트 케이크 출시
이미 존재 합니다.:두부와 크림치즈가 만났다 신세계푸드   강릉초당두부 케이크  출시
.이미 존재 합니다.:휠라코리아  신세계百 5개 점포에 케즈 매장 연다
이미 존재 합니다.:휠라 케즈 신세계 백화점 5개 매장 오픈
..이미 존재 합니다.:신세계 시코르  새내기 신입사원 대상  랜선 뷰티 클래스 
이미 존재 합니다.:신세계  새내기 신입사원 메이크업  시코르가 알려드려요 
.이미 존재 합니다.:신세계인터  에스아이빌리지서 신세계면세점 명품 재고 할인 판매
이미 존재 합니다.:신세계인터내셔날  구매한도 없는 면세점 재고 할인 판매
이미 존재 합니다.: